In [19]:
#package installs
!E:\Python310\python.exe -m pip install --upgrade pip
!E:\Python310\Scripts\pip3.exe install pip install moviepy pydub SpeechRecognition
!E:\Python310\python.exe -m pip install git+https://github.com/openai/whisper.git soundfile

  Cloning https://github.com/openai/whisper.git to c:\users\johnc\appdata\local\temp\pip-req-build-7yw9m1qp
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.0 MB 640.0 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.0 MB 648.1 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.0 MB 558.5 k

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\johnc\AppData\Local\Temp\pip-req-build-7yw9m1qp'


In [6]:
#Global variables
import socket
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = ''
ENV_FOLDER_DATA_PROC = ''
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

ENV_HOSTNAME:JWGamingPC
ENV_FOLDER_DATA:E:\W4732 Computer Vision\Final Paper Data\
ENV_PRATT:


In [13]:
# Take initial video
# using test video #568 Dr. Rhonda Patrick
# https://archive.org/download/jre-001-837/JRE_001-837/
# https://archive.org/download/jre-001-837/JRE_001-837/Joe%20Rogan%20Experience%20%23568%20-%20Dr.%20Rhonda%20Patrick.mp4

#take video and split into audio and video streams
import moviepy
import moviepy.editor

#extract audio into mp3
#https://medium.com/featurepreneur/extracting-audio-from-video-using-pythons-moviepy-library-e351cd652ab8

def split_mp4_to_mp3(filepath_mp4 , filepath_mp3):
    # Load the video clip
    video_clip = moviepy.editor.VideoFileClip(filepath_mp4)

    # Extract the audio from the video clip
    audio_clip = video_clip.audio

    # Write the audio to a separate file
    audio_clip.write_audiofile(filepath_mp3)

    # Close the video and audio clips
    audio_clip.close()
    video_clip.close()

filepath_testmp4 = ENV_FOLDER_DATA + 'Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4'
filepath_testmp3 = ENV_FOLDER_DATA_PROC + '568.mp3'
split_mp4_to_mp3(filepath_testmp4,filepath_testmp3)



MoviePy - Writing audio in E:\W4732 Computer Vision\Final Paper Data Proc\568.mp3


MoviePy - Done.


In [20]:
# Import the required libraries
import speech_recognition as sr  # Library for speech recognition
import os  # Library for interacting with the operating system
from pydub import AudioSegment  # Library for working with audio files
from pydub.silence import split_on_silence  # Function for splitting audio files based on silence

#https://stackoverflow.com/questions/65489705/transcribing-mp3-to-text-python-riff-id-error
recognizer = sr.Recognizer()

def transcribe_large_audio_file(prefix,path_mp3):
    """
    Split audio into chunks and apply speech recognition
    """
    # Load audio file with pydub
    audio = AudioSegment.from_mp3(path_mp3)
    # Split audio at silent parts with duration of 700ms or more and obtain chunks
    audio_chunks = split_on_silence(audio, min_silence_len=600, silence_thresh=audio.dBFS-14, keep_silence=600)

    # Create a directory to store audio chunks
    chunks_dir = ENV_FOLDER_DATA_PROC + prefix
    if not os.path.isdir(chunks_dir):
        os.mkdir(chunks_dir)

    full_text = ""
    failed_attempts = 0
    # Process each audio chunk
    for i, chunk in enumerate(audio_chunks, start=1):
        # Save chunk in the directory
        chunk_file_name = os.path.join(chunks_dir, f"chunk{i}.wav")
        chunk.export(chunk_file_name, format="wav")
        # Recognize audio from the chunk
        with sr.WavFile(chunk_file_name) as src:
            listened_audio = recognizer.listen(src)
            # Convert audio to text
            try:
                text = recognizer.recognize_whisper(listened_audio)
            except Exception  as e:
                failed_attempts += 1
                print(e)
            else:
                failed_attempts = 0
                text = f"{text.capitalize()}. "
                print(chunk_file_name, ":", text)
                full_text += text
    # Return the transcription for all chunks
    return full_text

def split_and_transcribe(prefix,filepath_mp3):
    # Define the output directory
    output_dir = ENV_FOLDER_DATA_PROC + prefix

    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate through all .mp3 files in the directory and transcribe them
    with open(os.path.join(output_dir, '568.txt'), 'w') as result_file:
        print(f"Processing {filepath_mp3}")
        try:
            # Transcribe the audio file
            transcription = transcribe_large_audio_file(prefix,filepath_mp3)
        except LookupError as error:
            # If there is an error, skip the file and continue with the next one
            print(f"Error on {filepath_mp3} due to: {error}")
        # Save the transcription to a text file with the same name as the audio file
        txt_file_path = os.path.join(output_dir, f"{os.path.splitext(filepath_mp3)[0]}.txt")
        with open(txt_file_path, 'w', encoding="utf-8") as txt_file:
            txt_file.write(transcription)
        # Print the transcription and the path to the saved text file
        print(transcription)
        print(f"Transcription saved to {txt_file_path}")
        # Save the transcription to the result


split_and_transcribe('568',filepath_testmp3)
#splitting file into pieces
#https://stackoverflow.com/questions/67334379/cut-mp4-in-pieces-python

Processing E:\W4732 Computer Vision\Final Paper Data Proc\568.mp3


100%|███████████████████████████████████████| 139M/139M [00:05<00:00, 24.6MiB/s]


E:\W4732 Computer Vision\Final Paper Data Proc\568\chunk1.wav :  the joe rogan experience train my nature, rogan podcast my night, all day doctor ronda, i presume. good to see you again. good to see you too. last time i saw you, i introduced you to the wonderful world of head trauma.. 
E:\W4732 Computer Vision\Final Paper Data Proc\568\chunk2.wav :  indeed. dr. ronda got her first experience of a live ufc event and. 
E:\W4732 Computer Vision\Final Paper Data Proc\568\chunk3.wav :  just to preface it you really didn't know much about the ufc at all before you went to this thing with your husband and it was. 
E:\W4732 Computer Vision\Final Paper Data Proc\568\chunk4.wav :  well, you're pretty shocked. yes, to be honest, i had no clue until.... 
E:\W4732 Computer Vision\Final Paper Data Proc\568\chunk5.wav :  i think i had been like the morning of, or the day before. and i was like the morning of the day before, i was like that.. 
E:\W4732 Computer Vision\Final Paper Data Proc\568\chunk6.

UnicodeEncodeError: 'charmap' codec can't encode characters in position 148901-148906: character maps to <undefined>

In [ ]:
#audio processing

In [ ]:
#video processing